## Tabla de contenidos 

1. [Contexto del modelo](#id1)

    1.1. [Librerías utilizadas](#id2)

2. [Preparación del dataset](#id3)
    
3. [Construcción del modelo](#id4)

4. [Predicciones del modelo](#id5)

5. [Evaluación](#id6)

    5.1. [Análisis por etiqueta](#id7)

    5.2. [Análisis general](#id8)

    5.3. [Conclusiones](#id9)

6. [Exportación del modelo](#id10)

<div id='id1' />

# 1. Contexto del modelo

El modelo en cuestión es un clasificador de texto que utiliza técnicas de procesamiento de lenguaje natural (NLP) para la tarea de clasificación multietiqueta. Más específicamente, utiliza un vectorizador TF-IDF (Term Frequency-Inverse Document Frequency) junto con un clasificador de regresión logística multinominal.

<div id='id2' />

## 1.1. Librerías utilizadas

In [2]:
# Tratamiento de datos

import numpy as np
import pandas as pd
import pickle
import warnings
warnings.filterwarnings('ignore')

# Gráficas

import seaborn as sns
import matplotlib as mpl 
import matplotlib.pyplot as plt

# Preprocesamiento y modelado

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

<div id='id3' />

# 2. Preparación del dataset

In [3]:
comments_df_undersampled = pd.read_csv('../01_data/02_processed/comments_df_undersampled.csv')
comments_df_undersampled

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,11ec6078ad3cd031,\n\nsee in my userpage the articles ive create...,0,0,0,0,0,0
1,11c0246ef55ef048,\n\nmatt fax page being deleted\n\nhi you just...,0,0,0,0,0,0
2,278ec4e59398a50b,this article contains information soursed from...,0,0,0,0,0,0
3,c2788f8bdaefeb13,24 promo \n\ni was really confused when i saw ...,0,0,0,0,0,0
4,fa04aa41f79c7884,i agree hes a biased editor 1141791837,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
32445,fef4cf7ba0012866,\n\n our previous conversation \n\nyou fucking...,1,0,1,0,1,1
32446,ff39a2895fc3b40e,you are a mischievious pubic hair,1,0,0,0,1,0
32447,ffa33d3122b599d6,your absurd edits \n\nyour absurd edits on gre...,1,0,1,0,1,0
32448,ffb47123b2d82762,\n\nhey listen dont you ever delete my edits e...,1,0,0,0,1,0


In [4]:
comments_df_undersampled.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [5]:
# Realizamos el train-test split

X_train, X_test, y_train, y_test = train_test_split(comments_df_undersampled['comment_text'], comments_df_undersampled[[ 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate']], test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(25960,)
(6490,)
(25960, 6)
(6490, 6)


<div id='id4' />

# 3. Construcción del modelo

In [11]:
# Instanciamos el TfidfVectorizer y del LogisticRegression multinominal con sus correspondientes parámetros

vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1, 2), max_features=500000)

rf_classifier = RandomForestClassifier(
    n_estimators=200,  # Número de árboles en el ensamble
    max_depth=50,      # Profundidad máxima de cada árbol
    min_samples_leaf=2,  # Número mínimo de muestras por hoja
    min_samples_split=5,  # Número mínimo de muestras para dividir un nodo
    max_features='sqrt',  # Máximo número de características consideradas para dividir
    bootstrap=True,    # Utilizar muestreo con reemplazo
    random_state=42    # Semilla aleatoria para reproducibilidad
)

multi_label_classifier = MultiOutputClassifier(rf_classifier, n_jobs=-1)

In [12]:
# Generamos el pipeline con las dos instancias anteriores y lo entrenamos

pipeline = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', multi_label_classifier)
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(max_features=500000, ngram_range=(1, 2),
                                 stop_words='english')),
                ('classifier',
                 MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=50,
                                                                        min_samples_leaf=2,
                                                                        min_samples_split=5,
                                                                        n_estimators=200,
                                                                        random_state=42),
                                       n_jobs=-1))])

<div id='id5' />

# 4. Predicciones del modelo

In [13]:
# Realizamos la predicción

y_pred = pipeline.predict(X_test)

<div id='id6' />

# 5. Evaluación del modelo

In [14]:
# Calculamos la matriz de confusión, precisión, recall y categorical accuracy para cada etiqueta

labels = comments_df_undersampled[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

for i, label in enumerate(labels.columns):
    print(f"Resultados para la etiqueta: {label}")
    print(f"Confusion Matrix:\n{confusion_matrix(y_test.iloc[:, i], y_pred[:, i])}")
    print(f"Precision: {precision_score(y_test.iloc[:, i], y_pred[:, i], zero_division=1)}")
    print(f"Recall: {recall_score(y_test.iloc[:, i], y_pred[:, i], zero_division=1)}")
    print(f"Categorical Accuracy: {accuracy_score(y_test.iloc[:, i], y_pred[:, i])}")
    print("-" * 50)

# Calculamos la precisión categórica general, o lo que es lo mismo, que todas las etiquetas deban coincidir

categorical_accuracy = accuracy_score(y_test, y_pred)
print(f"Categorical Accuracy General: {categorical_accuracy}")

Resultados para la etiqueta: toxic
Confusion Matrix:
[[3174  263]
 [ 789 2264]]
Precision: 0.8959240205777602
Recall: 0.7415656731084179
Categorical Accuracy: 0.837904468412943
--------------------------------------------------
Resultados para la etiqueta: severe_toxic
Confusion Matrix:
[[6191    1]
 [ 298    0]]
Precision: 0.0
Recall: 0.0
Categorical Accuracy: 0.9539291217257319
--------------------------------------------------
Resultados para la etiqueta: obscene
Confusion Matrix:
[[4786    0]
 [1651   53]]
Precision: 1.0
Recall: 0.031103286384976527
Categorical Accuracy: 0.7456086286594761
--------------------------------------------------
Resultados para la etiqueta: threat
Confusion Matrix:
[[6401    0]
 [  89    0]]
Precision: 1.0
Recall: 0.0
Categorical Accuracy: 0.9862865947611711
--------------------------------------------------
Resultados para la etiqueta: insult
Confusion Matrix:
[[4934    0]
 [1541   15]]
Precision: 1.0
Recall: 0.009640102827763496
Categorical Accuracy: 0

In [15]:
# Calcular y mostrar la precisión y el recall combinados
precision_micro = precision_score(y_test, y_pred, average='micro', zero_division=1)
recall_micro = recall_score(y_test, y_pred, average='micro', zero_division=1)
f1_micro = f1_score(y_test, y_pred, average='micro', zero_division=1)

precision_macro = precision_score(y_test, y_pred, average='macro', zero_division=1)
recall_macro = recall_score(y_test, y_pred, average='macro', zero_division=1)
f1_macro = f1_score(y_test, y_pred, average='macro', zero_division=1)

print(f"Precision (Micro): {precision_micro}")
print(f"Recall (Micro): {recall_micro}")
print(f"F1 Score (Micro): {f1_micro}")

print(f"Precision (Macro): {precision_macro}")
print(f"Recall (Macro): {recall_macro}")
print(f"F1 Score (Macro): {f1_macro}")

# Calcular y mostrar la precisión categórica general (es decir, todas las etiquetas deben coincidir)
categorical_accuracy = accuracy_score(y_test, y_pred)
print(f"Categorical Accuracy General: {categorical_accuracy}")

Precision (Micro): 0.8983050847457628
Recall (Micro): 0.33515377982178784
F1 Score (Micro): 0.48817249319656686
Precision (Macro): 0.8159873367629601
Recall (Macro): 0.130384843720193
F1 Score (Macro): 0.148482626551984
Categorical Accuracy General: 0.565331278890601


<div id='id7' />

## 5.1. Análisis por etiqueta

1. **Toxic**:
- **Precision**: 0.8959240205777602
- **Recall**: 0.7415656731084179
- **Categorical Accuracy**: 0.837904468412943
- **Interpretación**: La precisión indica que el 89.59% de las predicciones positivas para toxic fueron correctas, mientras que el recall muestra que se capturó el 74.16% de todos los verdaderos positivos de toxic. La precisión es relativamente alta, pero el recall podría mejorarse.

2. **Severe_toxic**:
- **Precision**: 0.0
- **Recall**: 0.0
- **Categorical Accuracy**: 0.9539291217257319
- **Interpretación**: La falta de precisión y recall indica que el modelo no identificó correctamente ninguna instancia de severe_toxic. Esto sugiere que el modelo tiene dificultades para clasificar esta clase específica y podría necesitar ajustes significativos.

3. **Obscene**:
- **Precision**: 1.0
- **Recall**: 0.031103286384976527
- **Categorical Accuracy**: 0.7456086286594761
- **Interpretación**: Aunque la precisión es perfecta, el recall es muy bajo, lo que indica que el modelo clasificó correctamente las pocas instancias que identificó como obscene, pero pasó por alto la mayoría de los casos positivos.

4. **Threat**:
- **Precision**: 1.0
- **Recall**: 0.0
- **Categorical Accuracy**: 0.9862865947611711
- **Interpretación**: Similar a severe_toxic, la precisión es alta pero el recall es nulo, lo que indica que el modelo no pudo identificar correctamente ninguna instancia de threat. Este resultado señala una necesidad urgente de mejorar la capacidad del modelo para detectar esta clase.

5. **Insult**:
- **Precision**: 1.0
- **Recall**: 0.009640102827763496
- **Categorical Accuracy**: 0.762557781201849
- **Interpretación**: La precisión es alta pero el recall es muy bajo, lo que sugiere que el modelo clasifica correctamente las instancias que detecta como insult, pero falla en identificar la mayoría de los casos positivos.

6. **Identity_hate**:
- **Precision**: 1.0
- **Recall**: 0.0
- **Categorical Accuracy**: 0.9602465331278891
- **Interpretación**: Al igual que severe_toxic y threat, la precisión es alta pero el recall es nulo, lo que indica una incapacidad del modelo para identificar correctamente las instancias de identity_hate.

<div id='id8' />

## 5.2. Análisis General

1. **Categorical Accuracy General**: 0.6553
   - Esto indica que solo el 65.5% de las predicciones coinciden completamente con las etiquetas verdaderas.

2. **Micro Average**:
   - **Precisión**: 0.8564
   - **Recall**: 0.6643
   - **F1 Score**: 0.7482
   - **Evaluación**: La precisión es bastante alta, pero el recall es menor, lo que sugiere que el modelo es más confiable en sus predicciones correctas que en la identificación exhaustiva de todas las etiquetas.

3. **Macro Average**:
   - **Precisión**: 0.8003
   - **Recall**: 0.4123
   - **F1 Score**: 0.4923
   - **Evaluación**: Estas métricas promedian el rendimiento en todas las etiquetas. La diferencia notable entre precisión y recall muestra que el modelo tiene dificultades con algunas etiquetas más que con otras.

1. **Micro-Precision**:
- **Valor**: 0.8983050847457628
- **Interpretación**: Esta métrica calcula la precisión global considerando todas las predicciones y verdaderos positivos, dividido por el total de predicciones. Indica que el 89.83% de las predicciones fueron correctas en el conjunto de datos total.

2. **Micro-Recall**:
- **Valor**: 0.33515377982178784
- **Interpretación**: Calcula el recall global considerando todos los verdaderos positivos y falsos negativos, dividido por el total de verdaderos positivos. Muestra que el 33.52% de las etiquetas positivas fueron correctamente identificadas en el conjunto de datos total.

3. **Micro-F1 Score**:
- **Valor**: 0.48817249319656686
- **Interpretación**: Es una medida que combina precisión y recall en una sola métrica. El puntaje F1 micro se calcula a partir de la precisión micro y el recall micro. Un valor más alto indica un mejor equilibrio entre precisión y recall en el conjunto de datos total.

4. **Macro-Precision**:
- **Valor**: 0.8159873367629601
- **Interpretación**: Esta métrica promedia la precisión de cada clase sin tener en cuenta el desbalance de clases. Indica que, en promedio, el modelo tiene una precisión del 81.60% para todas las clases consideradas individualmente.

5. **Macro-Recall**:
- **Valor**: 0.130384843720193
- **Interpretación**: Promedia el recall de cada clase sin considerar el desbalance de clases. Muestra que, en promedio, el modelo pudo recuperar el 13.04% de todas las instancias positivas.

6. **Macro-F1 Score**:
- **Valor**: 0.148482626551984
- **Interpretación**: Es la media armónica de la precisión macro y el recall macro. Un valor bajo indica un desempeño deficiente en el conjunto de datos total en términos de equilibrio entre precisión y recall.

7. **Categorical Accuracy General**:
- **Valor**: 0.565331278890601
- **Interpretación**: Es la precisión global promediada sobre todas las clases. Muestra que el modelo clasificó correctamente el 56.53% de las instancias en todas las clases.

<div id='id9' />

## 5.3. Conclusión

El modelo muestra buenos resultados en clases más frecuentes como toxic y obscene, pero enfrenta desafíos significativos en la detección de clases menos frecuentes como severe_toxic, threat, insult, y identity_hate.
Se observa una disparidad entre la precisión (alta en algunos casos) y el recall (a menudo bajo), lo que sugiere que el modelo podría beneficiarse de técnicas avanzadas de modelos más sofisticados, como redes neuronales recurrentes (RNNs) o transformers, que pueden capturar mejor las relaciones complejas entre palabras en los comentarios y mejorar la capacidad del modelo para detectar las etiquetas menos frecuentes de manera más efectiva.

<div id='id10' />

# 6. Exportación del modelo

In [16]:
# Definimos el nombre del archivo
filename = '06_quinto_modelo.pkl'

# Guardamos el pipeline en un archivo pickle

with open(filename, 'wb') as file:
    pickle.dump(pipeline, file)